In [ ]:
# You will need an open AI API key for this demo:
import os
os.environ["OPENAI_API_KEY"]="your API key"

In [ ]:
import utils
import antimatter as am
amr  = utils.create_or_load_domain()

In [ ]:
amr.add_fact_type(name="user_in_team", description="Records that a user is in a team", 
                  arguments={
                      "user":"the username",
                      "team":"the team name"
                  })
amr.delete_all_facts("user_in_team")
amr.add_fact("user_in_team", "bob", "sales")
amr.add_fact("user_in_team", "fred", "engineering");

In [ ]:
# Configure how we want data to be handled when it is written into a capsule.
amr.add_write_context(
    "kbase", am.WriteContextBuilder().
    set_summary("Data prepared for our knowledgebase").
    add_hook("regex-classifier", ">1.0.0", am.WriteContextHookMode.Sync).
    add_hook("data-structure-classifier", ">1.0.0", am.WriteContextHookMode.Sync)
);

In [ ]:
# Configure regular expression rules to apply tags to the entire container if the 
# team flag is found
amr.delete_write_context_regex_rules("kbase")
amr.insert_write_context_regex_rule(
    "kbase", 
    am.WriteContextRegexRuleBuilder("document-team:\s?sales").
        add_capsule_tag("tag.contoso.com/team", am.TagType.String, "sales")
)
amr.insert_write_context_regex_rule(
    "kbase", 
    am.WriteContextRegexRuleBuilder("document-team:\s?engineering").
        add_capsule_tag("tag.contoso.com/team", am.TagType.String, "engineering")
);

In [ ]:
# Configure how we want data to be handled when it is read for the purposes of calculating embeddings
amr.add_read_context(
    "kbase_embeddings", am.ReadContextBuilder().
    set_summary("The version of data to use when calculating embeddings")
)
# no rules: use data as-is

# Configure what the data looks like when loaded back by the chatbot:
amr.add_read_context(
    "kbase_chatbot", am.ReadContextBuilder().
    set_summary("Chatbot backend").
    add_read_parameter("user",True,"the user asking a question").
    add_required_hook("regex-classifier", ">1.0.0", "kbase")
)
amr.delete_read_context_rules("kbase_chatbot")
# Add a rule that says you can load documents belonging to the same team as the user
amr.add_read_context_rules("kbase_chatbot", rule_builder=am.ReadContextRuleBuilder().
    add_fact(
        name="user_in_team",
        operator=am.Operator.Exists,
        arguments_builder=am.ReadContextRuleFactArgumentBuilder().
            add_argument(source=am.Source.ReadParameters, key="user").
            add_argument(source=am.Source.Tags, key="tag.contoso.com/team")
    ).
    set_action(am.Action.Allow).
    set_priority(10)
)
# otherwise deny
amr.add_read_context_rules("kbase_chatbot", rule_builder=am.ReadContextRuleBuilder().
    set_action(am.Action.DenyRecord).
    set_priority(20)
);

In [ ]:
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema import Document
import json
import base64

In [ ]:
with open("notion.json","r") as f:
    content = json.load(f)
capsules = []
for d in [Document(page_content=p) for p in content]:
    cap = amr.encapsulate(d.page_content, write_context="kbase")
    capsules.append(cap)
    
docs_for_vectorizing = [amr.load_capsule(data=c, read_context="kbase_embeddings").data() for c in capsules]
vecs = OpenAIEmbeddings().embed_documents(docs_for_vectorizing)
kbase = list(zip([base64.b64encode(bytes(c)) for c in capsules], vecs))
faiss = FAISS.from_embeddings(kbase, OpenAIEmbeddings())

In [ ]:
retriever = utils.vector_retriever(amr, faiss.as_retriever(search_kwargs={"k": 2}), 
            read_context="kbase_chatbot", read_parameters={"user":"fred"})
chatbot = ConversationalRetrievalChain.from_llm(ChatOpenAI(model='gpt-4'), retriever=retriever)

resp = chatbot.invoke({'question': "how much did product B make in sales?", 'chat_history': ""})
print(resp["answer"])


In [ ]:
retriever = utils.vector_retriever(amr, faiss.as_retriever(search_kwargs={"k": 2}), 
            read_context="kbase_chatbot", read_parameters={"user":"bob"})
chatbot = ConversationalRetrievalChain.from_llm(ChatOpenAI(model='gpt-4'), retriever=retriever)

resp = chatbot.invoke({'question': "What is our production database password?", 'chat_history': ""})
print(resp["answer"])

In [ ]:
amr.add_fact("user_in_team","bob","engineering");